In [1]:
# Login
!az login > /dev/null

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CRBWYWG63 to authenticate.


In [3]:
# Connect to workspace
from azureml.core import Workspace
ws = Workspace.from_config()

In [24]:
# Run an experiment script
from azureml.core import Experiment, ScriptRunConfig, Environment

# Setup Docker environment
myenv = Environment(name="myenv")
myenv.docker.enabled = True
with open("Dockerfile", "r") as f:
    dockerfile=f.read()
myenv.docker.base_dockerfile = dockerfile
myenv.python.user_managed_dependencies = True

# Create a script config
script_config = ScriptRunConfig(source_directory='scripts',
                                script='diabetes_lr.py',
                                arguments = ['--reg-rate', 0.1],
                                compute_target=ws.compute_targets['instance-target'],
                                environment=myenv) 

# Submit the experiment
experiment = Experiment(workspace=ws, name='diabetes_lr')
run = experiment.submit(config=script_config)
# run.wait_for_completion()

Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None


In [25]:
# List the files generated by the experiment
for file in run.get_file_names():
    print(file)

azureml-logs/20_image_build_log.txt
azureml-logs/55_azureml-execution-tvmps_93d8168dbcc30cfc012d53f3aa7fd3c546329757ddd3bc44b62a3dcb15ee798b_d.txt
azureml-logs/65_job_prep-tvmps_93d8168dbcc30cfc012d53f3aa7fd3c546329757ddd3bc44b62a3dcb15ee798b_d.txt
azureml-logs/70_driver_log.txt
azureml-logs/75_job_post-tvmps_93d8168dbcc30cfc012d53f3aa7fd3c546329757ddd3bc44b62a3dcb15ee798b_d.txt
azureml-logs/process_info.json
azureml-logs/process_status.json
logs/azureml/91_azureml.log
logs/azureml/dataprep/python_span_e66919f2-24ea-41de-945a-321483e1ce2c.jsonl
logs/azureml/dataprep/python_span_ebeede72-9921-4259-8776-4495075ca792.jsonl
logs/azureml/job_prep_azureml.log
logs/azureml/job_release_azureml.log
outputs/diabetes.pkl


In [26]:
# Register model
from azureml.core import Model

run.register_model( model_name='diabetes_model',
                    model_path='outputs/diabetes.pkl', # run outputs path
                    description='A diabetes classification model',
                    tags={'data-format': 'CSV'},
                    model_framework=Model.Framework.SCIKITLEARN,
                    model_framework_version='0.20.3')

Model(workspace=Workspace.create(name='dscert', subscription_id='1bd4017e-d44c-46ab-9e6d-400b8d5f3da8', resource_group='ds_cert'), name=diabetes_model, id=diabetes_model:1, version=1, tags={'data-format': 'CSV'}, properties={})

In [27]:
# View the model
from azureml.core import Model

for model in Model.list(ws):
    # Get model name and auto-generated version
    print(model.name, 'version:', model.version)

diabetes_model version: 1
